In [ ]:
#@title Import Dependencies

import torch
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms
from torch.autograd import Variable
import itertools
import pickle
import numpy as np
from numpy import random
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


In [ ]:
from tqdm import tqdm
from PIL import Image
from matplotlib import pyplot as plt
import matplotlib
from typing import TypeVar, Dict

import torch
import torch.nn as nn
from torch.optim.optimizer import Optimizer

from counterfactuals.utils import make_dir, get_transforms, torch_to_image, expl_to_image
from counterfactuals.plot import plot_grid_part
from counterfactuals.generative_models.base import GenerativeModel
from counterfactuals.classifiers.base import NeuralNet

Tensor = TypeVar('torch.tensor')

#matplotlib.use('Agg')

import click
from argparse import Namespace
import ast
import os

import torchvision.transforms as transforms
from torch.autograd import Variable
from counterfactuals.classifiers.base import NeuralNet
import torch.nn.functional as F
from typing import TypeVar, Tuple
import counterfactuals.classifiers.cnn as classifiers
import counterfactuals.classifiers.unet as unet
from counterfactuals.utils import load_checkpoint
from counterfactuals.data import get_data_info
from counterfactuals.generative_models.factory import get_generative_model
import gdown
import copy

In [ ]:
#@title Define Hyperparameters

# class_names = ["5_o_Clock_Shadow", "Arched_Eyebrows", "Attractive", "Bags_Under_Eyes", "Bald", "Bangs",
#                 "Big_Lips", "Big_Nose", "Black_Hair", "Blond_Hair", "Blurry", "Brown_Hair", "Bushy_Eyebrows",
#                 "Chubby", "Double_Chin", "Eyeglasses", "Goatee", "Gray_Hair", "Heavy_Makeup", "High_Cheekbones",
#                 "Male", "Mouth_Slightly_Open", "Mustache", "Narrow_Eyes", "No_Beard", "Oval_Face", "Pale_Skin",
#                 "Pointy_Nose", "Receding_Hairline", "Rosy_Cheeks", "Sideburns", "Smiling", "Straight_Hair",
#                 "Wavy_Hair", "Wearing_Earrings", "Wearing_Hat", "Wearing_Lipstick", "Wearing_Necklace",
#                 "Wearing_Necktie", "Young"]
# num_classes = 40
# data_shape = [3, 64, 64]
# n_bits = 5
# temp = 0.7
# data_mean = [0.485, 0.456, 0.406]
# data_std = [0.229, 0.224, 0.225]

input_size = 28*28*1 # img_size = (28,28) ---> 28*28=784 in total
batch_size = 200 # the size of input data took for one iteration

In [ ]:
# from torchvision.transforms import v2
# transform_RandomErasing=transforms.Compose([v2.RandomErasing(),
#                               transforms.ToTensor()])
#from torchvision.transforms import v2
#transform=transforms.Compose([transforms.ToTensor(),transforms.Normalize(mean=0,std=1.0)])
transform=transforms.Compose([transforms.ToTensor()])
train_data = dsets.FashionMNIST(root = './data', train=True, transform = transform, download = True)
test_data = dsets.FashionMNIST(root = './data', train=False, transform = transform, download = True)

In [ ]:
#@title Loading the data

train_gen = torch.utils.data.DataLoader(dataset = train_data,
                                             batch_size = batch_size,
                                             shuffle = True)

test_gen = torch.utils.data.DataLoader(dataset = test_data,
                                      batch_size = batch_size,
                                      shuffle = False)



In [ ]:
#cvmx=torch.zeros([3*64*64,3*64*64],device=device)
images_list=[]
labels_list=[]
no_batches=len(train_gen)
#images_mean=torch.zeros(3,64,64,device=device)
for i ,(images,labels) in enumerate(train_gen):
    images = Variable(images).cuda().detach()
    labels=Variable(labels).cuda().detach()
    # images_mean=images_mean+images.mean(0)
    # im=torch.reshape(images,[images.shape[0],3*64*64])
    # cvmx+=torch.matmul(torch.transpose(im,0,1),im)
    if(i<(len(train_gen))):
        images_list.append(images)
        labels_list.append(labels)



test_images_list=[]
test_labels_list=[]
test_no_batches=len(test_gen)
for i ,(images,labels) in enumerate(test_gen):
    images = Variable(images).cuda().detach()
    labels=Variable(labels).cuda().detach()
    if(i<(len(test_gen))):
        test_images_list.append(images)
        test_labels_list.append(labels)

train_data_len=len(train_data)
test_data_len=len(test_data)



In [ ]:
#@title Define model class
import torch.nn as nn
import torch
import torch.nn.functional as F
from typing import TypeVar, Tuple

Tensor = TypeVar('torch.tensor')

class NeuralNet(torch.nn.Module):
    """
    base class for all NN classifiers
    """
    def __init__(self):
        super().__init__()

    def initialize_weights(self):
        for m in self.modules():
            if isinstance(m, torch.nn.Conv2d):
                torch.nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
                if m.bias is not None:
                    torch.nn.init.constant_(m.bias, 0)
            elif isinstance(m, torch.nn.BatchNorm2d):
                torch.nn.init.constant_(m.weight, 1)
                torch.nn.init.constant_(m.bias, 0)
            elif isinstance(m, torch.nn.Linear):
                torch.nn.init.normal_(m.weight, 0, 0.01)
                torch.nn.init.constant_(m.bias, 0)

class multinomial(NeuralNet):
    """
    CNN for (binary) classification for CelebA, CheXpert
    """

    def __init__(self,
                 num_classes: int = 10,
                 flattened_size: int = 28*28):
        """Builder."""
        super(multinomial, self).__init__()

        self.last_layer=nn.Sequential(
            nn.Flatten(),
            nn.Linear(flattened_size, num_classes)
        )

    def forward(self, x: Tensor) -> Tensor:
        """Perform forward."""

        x=self.last_layer(x)

        return x

    def classify(self, x: Tensor) -> Tuple[Tensor, Tensor, Tensor]:
        net_out = self.forward(x)
        acc = F.softmax(net_out, dim=1)
        class_idx = torch.max(net_out, 1)[1]

        return acc, acc[0, class_idx], class_idx
    




In [ ]:
# net=Fashion_MNIST_CNN()
# n=0
# for par in net.parameters():
#     n+=par.numel()

# n

In [ ]:
#@title Define loss-function & optimizer
loss_function = nn.CrossEntropyLoss()


def images_regulariser(net): 
    li_reg_loss = 0
    penalized     = [p for name,p in net.named_parameters() if 'bias' not in name]
    not_penalized = [p for name,p in net.named_parameters() if 'bias' in name]
    for p in penalized:
        li_reg_loss += (p**2).sum()*0.5
    #for p in net.parameters():
#        li_reg_loss += (p**2).sum()*0.5
    reg=li_reg_loss/(train_data_len)*l2regconst
    return(reg)


In [ ]:
def addnet(net,net2):
    for param1, param2 in zip(net.parameters(), net2.parameters()):
     param1.data += param2.data

def multiplynet(net,a):
   for param1 in net.parameters():
     param1.data *=a

In [ ]:
from dataclasses import dataclass
@dataclass
class hclass:
    h: Tensor
    eta: Tensor
    etam1g: Tensor
    c11: Tensor
    c21: Tensor
    c22: Tensor

@dataclass
class BAOABhclass:
    h: Tensor
    eta: Tensor
    xc1: Tensor
    xc2: Tensor
    xc3: Tensor
    vc1: Tensor
    vc2: Tensor
    vc3: Tensor


def hper2const(h,gam):
    gh=gam.double()*h.double()
    s=torch.sqrt(4*torch.expm1(-gh/2)-torch.expm1(-gh)+gh)
    eta=(torch.exp(-gh/2)).float()
    etam1g=((-torch.expm1(-gh/2))/gam.double()).float()
    c11=(s/gam).float()
    c21=(torch.exp(-gh)*(torch.expm1(gh/2.0))**2/s).float()
    c22=(torch.sqrt(8*torch.expm1(-gh/2)-4*torch.expm1(-gh)-gh*torch.expm1(-gh))/s).float()
    hc=hclass(h=h,eta=eta,etam1g=etam1g,c11=c11,c21=c21,c22=c22)
    return(hc)

def BAOAB_hconst(h,gam):
    with torch.no_grad():
        hh=copy.deepcopy(h).detach().double()
        gamm=copy.deepcopy(gam).detach().double()
        gh=gamm*hh
        eta=(torch.exp(-gh/2))
        xc1=hh/2*(1+eta)
        xc2=(hh*hh/4)*(1+eta)
        xc3=hh/2*torch.sqrt(-torch.expm1(-gh))
        vc1=eta*(hh/2)
        vc2=(hh/2)
        vc3=torch.sqrt(-torch.expm1(-gh))

        hc=BAOABhclass(h=hh.float(),eta=eta.float(),xc1=xc1.float(),xc2=xc2.float(),xc3=xc3.float(),vc1=vc1.float(),vc2=vc2.float(),vc3=vc3.float())
        return(hc)


def U(x,v,hc,xi1,xi2):

    xn=x+hc.etam1g*v+hc.c11*xi1
    vn=v*hc.eta+hc.c21*xi1+hc.c22*xi2
    return([xn, vn])

def bounce(x,v,xstar,width):
    vsign=(((x-xstar+width)/(2*width)).floor()% 2)*(-2)+1
    vn=v*vsign
    xn=((x-xstar-width)% (4*width)-2*width).abs()-width+xstar  
    return([xn, vn])

def bouncenet():
    for p,p_star in zip(net.parameters(),net_star.parameters()):
        [p.data, p.v]=bounce(p.data, p.v, p_star.data, 6/torch.sqrt(l2regconst_extra))

def svrg_grad(net, batch_it):
    outputsU = net(images_list[batch_it])
    loss_likelihood = loss_function(outputsU, labels_list[batch_it])  


    grads_reg=[torch.zeros_like(par) for par in net.parameters()]
    net_pars=list(net.parameters())
    with torch.no_grad():
        for it in range(len_params):
            if(list_no_bias[it]):
                grads_reg[it]=net_pars[it].data*l2regconst

    net.zero_grad()
    loss_likelihood.backward()
    with torch.no_grad():
        grads_likelihood=[par.grad*batch_size for par in net.parameters()]

        svrg_grads=[]
        for p,grad,grad_reg,p_star,grad_star,star_sum_grad in zip(list(net.parameters()),grads_likelihood,grads_reg,list(net_star.parameters()),net_star_grad_list[batch_it],net_star_full_grad):              
            svrg_grads.append(grad_reg+star_sum_grad+(grad-grad_star)*no_batches+l2regconst_extra*(p.data-p_star.data))
    return svrg_grads,loss_likelihood.data


def UBU_step(hper2c,batch_it):   
    with torch.no_grad():
        for p in list(net.parameters()):
            xi1=torch.randn_like(p.data,device=device)
            xi2=torch.randn_like(p.data,device=device)
            [p.data,p.v]=U(p.data,p.v,hper2c,xi1,xi2)


    svrg_grads,loss_likelihood_data=svrg_grad(net, batch_it)

    with torch.no_grad():

        for p,grad in zip(net.parameters(), svrg_grads):              
            #Using variance reduction
            p.v-=hper2c.h*grad
      
        for p in list(net.parameters()):
            xi1=torch.randn_like(p.data,device=device)
            xi2=torch.randn_like(p.data,device=device)
            [p.data,p.v]=U(p.data,p.v,hper2c,xi1,xi2)

    return(loss_likelihood_data)



def UBU_step2(net, net2, hper4c,batch_it_list):   
    with torch.no_grad():
        for p,q in zip(net.parameters(),net2.parameters()):
            xi1=torch.randn_like(p.data,device=device)
            xi2=torch.randn_like(p.data,device=device)
            [p.data,p.v]=U(p.data,p.v,hper4c,xi1,xi2)
            [q.data,q.v]=U(q.data,q.v,hper4c,xi1,xi2)

    svrg_grads2,_=svrg_grad(net2, batch_it_list[0])
   
    with torch.no_grad():
        for q,gradq in zip(net2.parameters(), svrg_grads2):              
            q.v-=hper4c.h*gradq

    with torch.no_grad():
        for p,q in zip(net.parameters(),net2.parameters()):
            xi1=torch.randn_like(p.data,device=device)
            xi2=torch.randn_like(p.data,device=device)
            [p.data,p.v]=U(p.data,p.v,hper4c,xi1,xi2)
            [q.data,q.v]=U(q.data,q.v,hper4c,xi1,xi2)

    svrg_grads,loss_likelihood_data=svrg_grad(net, batch_it_list[2])

    with torch.no_grad():
        for p,grad in zip(net.parameters(), svrg_grads):              
            p.v-=2*hper4c.h*grad

    with torch.no_grad():
        for p,q in zip(net.parameters(),net2.parameters()):
            xi1=torch.randn_like(p.data,device=device)
            xi2=torch.randn_like(p.data,device=device)
            [p.data,p.v]=U(p.data,p.v,hper4c,xi1,xi2)
            [q.data,q.v]=U(q.data,q.v,hper4c,xi1,xi2)

    svrg_grads2,_=svrg_grad(net2, batch_it_list[1])

    with torch.no_grad():
        for q,grad in zip(net2.parameters(), svrg_grads2):              
            q.v-=hper4c.h*grad

    with torch.no_grad():
        for p,q in zip(net.parameters(),net2.parameters()):
            xi1=torch.randn_like(p.data,device=device)
            xi2=torch.randn_like(p.data,device=device)
            [p.data,p.v]=U(p.data,p.v,hper4c,xi1,xi2)
            [q.data,q.v]=U(q.data,q.v,hper4c,xi1,xi2)
    
    return(loss_likelihood_data)

def EM_step2(net, net2, h, gam, batch_it_list):   
    svrg_grads2,_=svrg_grad(net2, batch_it_list[0])
    svrg_grads,loss_likelihood_data=svrg_grad(net, batch_it_list[2])   
    sqrt2=torch.tensor(2).sqrt().detach()

    with torch.no_grad():
        for p,gradp in zip(net2.parameters(), svrg_grads2):              
            p.xi=torch.randn_like(p.data,device=device)
            p.data+=p.v*h/2
            p.v-=(h/2)*gradp+gam*(h/2)*p.v-torch.sqrt(2*gam*h/2)*p.xi

    svrg_grads2,_=svrg_grad(net2, batch_it_list[1])

    with torch.no_grad():
        for p,gradp in zip(net2.parameters(), svrg_grads2):              
            p.xi2=torch.randn_like(p.data,device=device)
            p.data+=p.v*h/2
            p.v-=(h/2)*gradp+gam*(h/2)*p.v-torch.sqrt(2*gam*h/2)*p.xi2

        for p,q,gradp in zip(net.parameters(),net2.parameters(), svrg_grads):              
            p.data+=p.v*h
            p.v-=(h)*gradp+gam*(h)*p.v-torch.sqrt(2*gam*h)*(q.xi+q.xi2)/sqrt2


    return(loss_likelihood_data)


def BAOAB_step(net,hc,batch_it,last_grad):   
    
    #xn=x+xc1*v-xc2*grad+xc3*xip
    with torch.no_grad():
        for p,grad in zip(net.parameters(), last_grad):
            p.xi=torch.randn_like(p.data,device=device)
            p.data=p.data+hc.xc1*p.v-hc.xc2*grad+hc.xc3*p.xi

    svrg_grads,loss_likelihood_data=svrg_grad(net, batch_it)

    with torch.no_grad():
        for p,grad,gradn in zip(net.parameters(), last_grad,svrg_grads):              
            p.v=hc.eta*p.v-hc.vc1*grad-hc.vc2*gradn+hc.vc3*p.xi

    return(loss_likelihood_data,svrg_grads)



def BAOAB_step2(net, net2, hc, hper2c, batch_it_list,last_grad,last_grad2):   

    with torch.no_grad():
        for p,grad in zip(net2.parameters(), last_grad2):
            p.xi=torch.randn_like(p.data,device=device)
            p.data=p.data+hper2c.xc1*p.v-hper2c.xc2*grad+hper2c.xc3*p.xi

    svrg_grads2,_=svrg_grad(net2, batch_it_list[0])

    with torch.no_grad():
        for p,grad,gradn in zip(net2.parameters(), last_grad2,svrg_grads2):              
            p.v=hper2c.eta*p.v-hper2c.vc1*grad-hper2c.vc2*gradn+hper2c.vc3*p.xi

    with torch.no_grad():
        for p,grad in zip(net2.parameters(), svrg_grads2):
            p.xi2=torch.randn_like(p.data,device=device)
            p.data=p.data+hper2c.xc1*p.v-hper2c.xc2*grad+hper2c.xc3*p.xi2

    svrg_grads2n,_=svrg_grad(net2, batch_it_list[1])

    with torch.no_grad():
        for p,grad,gradn in zip(net2.parameters(), svrg_grads2,svrg_grads2n):              
            p.v=hper2c.eta*p.v-hper2c.vc1*grad-hper2c.vc2*gradn+hper2c.vc3*p.xi2

    sqrt2=torch.tensor(2).sqrt().detach()
    with torch.no_grad():
        for p,q,grad in zip(net.parameters(),net2.parameters(), last_grad):
            p.data=p.data+hc.xc1*p.v-hc.xc2*grad+hc.xc3*(q.xi+q.xi2)/sqrt2

    svrg_grads,loss_likelihood_data=svrg_grad(net, batch_it_list[2])

    with torch.no_grad():
        for p,q,grad,gradn in zip(net.parameters(),net2.parameters(), last_grad,svrg_grads):              
            p.v=hc.eta*p.v-hc.vc1*grad-hc.vc2*gradn+hc.vc3*(q.xi+q.xi2)/sqrt2

    return(loss_likelihood_data,svrg_grads,svrg_grads2n)


def ind_create(batch_it):
    modit=batch_it %(2*no_batches)
    ind=(modit<=(no_batches-1))*modit+(modit>=no_batches)*(2*no_batches-modit-1)
    return ind


In [ ]:
#net = Fashion_MNIST_CNN().cuda()
#net2=copy.deepcopy(net)
#addnet(net2,net)
#multiplynet(net2,1/10)

In [ ]:
# filepath="output_fashion_low_rank_9n.pickle"
# #filepath="output_fashion_low_rank_long.pickle"
# with open(filepath,"rb") as file:
#    [labels_arr,test_labels_arr,test_prob_arr]=pickle.load(file)
# labels_arr=torch.tensor(labels_arr).detach()
# test_labels_arr=torch.tensor(test_labels_arr).detach()
# test_prob_arr=torch.tensor(test_prob_arr).detach()

In [ ]:


#@title Output arrays
num_classes=10

training_size=no_batches*batch_size
test_size=test_data_len


l2regconst=torch.tensor(50).detach()
l2regconst_extra=torch.tensor(0).detach()
gam=torch.sqrt(l2regconst)
#hper2c=hper2const(torch.tensor(h/2),gam)
all_images=torch.cat(images_list,dim=0).detach()
all_labels=torch.cat(labels_list,dim=0).detach()
# net = multinomial().cuda()
# net2 = multinomial().cuda()
# net.train()
# net2.train()

def find_MAP(num_steps):
  net = multinomial().cuda()

  def lpost():
    outputs = net(all_images)    
    loss_likelihood = loss_function(outputs, all_labels)
    reg=images_regulariser(net)
    loss=loss_likelihood+reg
    optimizer.zero_grad()
    loss.backward()
    return(loss)

  optimizer = torch.optim.LBFGS(net.parameters(), history_size=30, max_iter=20)
  for epoch in range(num_steps):
    optimizer.step(lpost)
  
  net_star=copy.deepcopy(net)
  len_params=len(list(net_star.parameters()))
  #Variance reduction - saving gradients at each batch at x_star
  net_star_grad_list=[]
  net_star_full_grad=[torch.zeros_like(par, device=device) for par in list(net_star.parameters())]
  for i in range(no_batches):
      images=images_list[i]
      labels=labels_list[i]
      outputs=net_star(images)
      loss_likelihood = loss_function(outputs, labels)
      reg=images_regulariser(net)
      net_star.zero_grad()
      loss_likelihood.backward()
      grads=[par.grad*batch_size for par in list(net_star.parameters())]
      net_star_grad_list.append(grads)
      for g, gi in zip(net_star_full_grad,grads):
        g+=gi          

  len_params=len(list(net_star.parameters()))
  list_no_bias=torch.zeros(len_params)
  pit=0
  for name, p in net_star.named_parameters():
      if 'bias' not in name:
          list_no_bias[pit]=1.0
      pit+=1
  return net_star, net_star_grad_list, net_star_full_grad, len_params, list_no_bias

  

def SMS_UBU2(num_epochs,h,gam):
  net=copy.deepcopy(net_star)
  net2=copy.deepcopy(net_star)

  with torch.no_grad():
    hper4c=hper2const(torch.tensor(h/2),gam)
    test_labels_arr=torch.zeros(test_size).detach()
    test_prob_arr=torch.zeros([test_size,num_classes,num_epochs]).detach()
    test_prob_arr2=torch.zeros([test_size,num_classes,num_epochs]).detach()  
  #Initialise velocities
    for par,par2 in zip(net.parameters(), net2.parameters()):
      par.v = torch.randn_like(par,device=device).detach()
      par2.v=copy.deepcopy(par.v).detach()

  for epoch in range(num_epochs):
    sum_loss=0
    if(epoch%2==0):
      rperm=random.permutation(list(range(no_batches)))      
    rperm2=random.permutation(list(range(no_batches)))
    for i in range(no_batches):
      it=epoch*no_batches+i
      
      ind=ind_create(2*it)
      ind2=ind_create(2*it+1)            
      indc=ind_create(it)          
      batch_it_list=[rperm2[ind], rperm2[ind2], rperm[indc]]

      loss_likelihood=UBU_step2(net,net2,hper4c,batch_it_list)
      sum_loss=sum_loss+loss_likelihood

    print('Epoch [%d/%d], Step [%d/%d]' %(epoch+1, num_epochs, i+1, no_batches))
    correct = 0
    total = 0  

    with torch.no_grad():
      for testit in range(test_no_batches):
        imagest=test_images_list[testit]
        labelst=test_labels_list[testit]
        actual_batch_size=len(imagest)
        test_labels_arr[(testit*batch_size):(testit*batch_size+actual_batch_size)]=labelst.detach().cpu()
        outputt = net(imagest).detach()#.reshape(actual_batch_size).detach()
        outputt2= net2(imagest).detach()
        _, predictedt = torch.max(outputt,1)
        #_, predictedt2 = torch.max(outputt2,1)
        correct += (predictedt == labelst).sum()
        total += labelst.size(0)

        test_prob_arr[(testit*batch_size):(testit*batch_size+actual_batch_size),0:num_classes,epoch]=torch.softmax(outputt,dim=1)

        test_prob_arr2[(testit*batch_size):(testit*batch_size+actual_batch_size),0:num_classes,epoch]=torch.softmax(outputt2,dim=1)
    
    #net.train()       
    print('Test accuracy of the model: %.3f %%' %((100*correct)/(total+1)))
    print('Epoch [%d], Average Loss: %0.4f' %(epoch+1, sum_loss/no_batches))
  return(test_labels_arr,test_prob_arr,test_prob_arr2)
  
def SG_UBU2(num_epochs,h,gam):
  net=copy.deepcopy(net_star)
  net2=copy.deepcopy(net_star)
  with torch.no_grad():
    hper4c=hper2const(torch.tensor(h/2),gam)
    test_labels_arr=torch.zeros(test_size)
    test_prob_arr=torch.zeros([test_size,num_classes,num_epochs])
    test_prob_arr2=torch.zeros([test_size,num_classes,num_epochs])  
  #Initialise velocities
    for par,par2 in zip(net.parameters(), net2.parameters()):
      par.v = torch.randn_like(par,device=device).detach()
      par2.v=copy.deepcopy(par.v).detach()


  for epoch in range(num_epochs):
    sum_loss=0
    for i in range(no_batches):
      ind=torch.randint(high=no_batches,size=(1,1))
      ind2=torch.randint(high=no_batches,size=(1,1))
      if(torch.rand(1)<0.5):
        indc=ind
      else:
        indc=ind2
      batch_it_list=[ind, ind2, indc]

      loss_likelihood=UBU_step2(net,net2,hper4c,batch_it_list)
      sum_loss=sum_loss+loss_likelihood

    print('Epoch [%d/%d], Step [%d/%d]' %(epoch+1, num_epochs, i+1, no_batches))
    correct = 0
    total = 0  

    with torch.no_grad():
      for testit in range(test_no_batches):
        imagest=test_images_list[testit]
        labelst=test_labels_list[testit]
        actual_batch_size=len(imagest)
        test_labels_arr[(testit*batch_size):(testit*batch_size+actual_batch_size)]=labelst.detach().cpu()
        outputt = net(imagest).detach()#.reshape(actual_batch_size).detach()
        outputt2= net2(imagest).detach()
        _, predictedt = torch.max(outputt,1)
        #_, predictedt2 = torch.max(outputt2,1)
        correct += (predictedt == labelst).sum()
        total += labelst.size(0)

        test_prob_arr[(testit*batch_size):(testit*batch_size+actual_batch_size),0:num_classes,epoch]=torch.softmax(outputt,dim=1)

        test_prob_arr2[(testit*batch_size):(testit*batch_size+actual_batch_size),0:num_classes,epoch]=torch.softmax(outputt2,dim=1)
    
    #net.train()       
    print('Test accuracy of the model: %.3f %%' %((100*correct)/(total+1)))
    print('Epoch [%d], Average Loss: %0.4f' %(epoch+1, sum_loss/no_batches))
  return(test_labels_arr,test_prob_arr,test_prob_arr2)

def SG_EM2(num_epochs,h,gam):
  net=copy.deepcopy(net_star)
  net2=copy.deepcopy(net_star)
  with torch.no_grad():
    test_labels_arr=torch.zeros(test_size)
    test_prob_arr=torch.zeros([test_size,num_classes,num_epochs])
    test_prob_arr2=torch.zeros([test_size,num_classes,num_epochs])  
  #Initialise velocities
    for par,par2 in zip(net.parameters(), net2.parameters()):
      par.v = torch.randn_like(par,device=device).detach()
      par2.v=copy.deepcopy(par.v).detach()

  for epoch in range(num_epochs):
    sum_loss=0
    for i in range(no_batches):
      ind=torch.randint(high=no_batches,size=(1,1))
      ind2=torch.randint(high=no_batches,size=(1,1))
      if(torch.rand(1)<0.5):
        indc=ind
      else:
        indc=ind2
      batch_it_list=[ind, ind2, indc]

      loss_likelihood=EM_step2(net,net2,h,gam,batch_it_list)
      sum_loss=sum_loss+loss_likelihood

    print('Epoch [%d/%d], Step [%d/%d]' %(epoch+1, num_epochs, i+1, no_batches))
    correct = 0
    total = 0  

    with torch.no_grad():
      for testit in range(test_no_batches):
        imagest=test_images_list[testit]
        labelst=test_labels_list[testit]
        actual_batch_size=len(imagest)
        test_labels_arr[(testit*batch_size):(testit*batch_size+actual_batch_size)]=labelst.detach().cpu()
        outputt = net(imagest).detach()#.reshape(actual_batch_size).detach()
        outputt2= net2(imagest).detach()
        _, predictedt = torch.max(outputt,1)
        #_, predictedt2 = torch.max(outputt2,1)
        correct += (predictedt == labelst).sum()
        total += labelst.size(0)

        test_prob_arr[(testit*batch_size):(testit*batch_size+actual_batch_size),0:num_classes,epoch]=torch.softmax(outputt,dim=1)

        test_prob_arr2[(testit*batch_size):(testit*batch_size+actual_batch_size),0:num_classes,epoch]=torch.softmax(outputt2,dim=1)
    
    #net.train()       
    print('Test accuracy of the model: %.3f %%' %((100*correct)/(total+1)))
    print('Epoch [%d], Average Loss: %0.4f' %(epoch+1, sum_loss/no_batches))
  return(test_labels_arr,test_prob_arr,test_prob_arr2)

def SG_BAOAB(num_epochs,h,gam):
  net=copy.deepcopy(net_star)
  
  with torch.nograd():
    hc=BAOAB_hconst(h,gam)
    test_labels_arr=torch.zeros(test_size)
    test_prob_arr=torch.zeros([test_size,num_classes,num_epochs])
    #Initialise velocities
    for par in list(net.parameters()):
      par.v = torch.randn_like(par,device=device)      

  
  for epoch in range(num_epochs):
    sum_loss=0
    for i in range(no_batches):
      ind=torch.randint(high=no_batches,size=(1,1))
      if(epoch==0 and i==0):
        last_grad,_=svrg_grad(net,ind)
      loss_likelihood,last_grad=BAOAB_step(net,hc,ind,last_grad)

      sum_loss=sum_loss+loss_likelihood

    print('Epoch [%d/%d], Step [%d/%d]' %(epoch+1, num_epochs, i+1, no_batches))
    correct = 0
    total = 0  

    with torch.no_grad():
      for testit in range(test_no_batches):
        imagest=test_images_list[testit]
        labelst=test_labels_list[testit]
        actual_batch_size=len(imagest)
        test_labels_arr[(testit*batch_size):(testit*batch_size+actual_batch_size)]=labelst.detach().cpu()
        outputt = net(imagest).detach()#.reshape(actual_batch_size).detach()
        _, predictedt = torch.max(outputt,1)
        #_, predictedt2 = torch.max(outputt2,1)
        correct += (predictedt == labelst).sum()
        total += labelst.size(0)

        test_prob_arr[(testit*batch_size):(testit*batch_size+actual_batch_size),0:num_classes,epoch]=torch.softmax(outputt,dim=1)
    
    #net.train()       
    print('Test accuracy of the model: %.3f %%' %((100*correct)/(total+1)))
    print('Epoch [%d], Average Loss: %0.4f' %(epoch+1, sum_loss/no_batches))
  return(test_labels_arr,test_prob_arr)



def SG_BAOAB2(num_epochs,h,gam):
  net=copy.deepcopy(net_star)
  net2=copy.deepcopy(net_star)  
  with torch.no_grad():
    hc=BAOAB_hconst(h,gam)
    hper2c=BAOAB_hconst(h/2,gam)      
    test_labels_arr=torch.zeros(test_size)
    test_prob_arr=torch.zeros([test_size,num_classes,num_epochs])
    test_prob_arr2=torch.zeros([test_size,num_classes,num_epochs])  
  #Initialise velocities
    for par,par2 in zip(net.parameters(), net2.parameters()):
      par.v = torch.randn_like(par,device=device).detach()
      par2.v=copy.deepcopy(par.v).detach()
  
  for epoch in range(num_epochs):
    sum_loss=0
    for i in range(no_batches):

      ind=torch.randint(high=no_batches,size=(1,1))
      ind2=torch.randint(high=no_batches,size=(1,1))
      if(torch.rand(1)<0.5):
        indc=ind
      else:
        indc=ind2
      batch_it_list=[ind, ind2, indc]

      if(epoch==0 and i==0):
        last_grad,_=svrg_grad(net,ind)
        last_grad2,_=svrg_grad(net2,indc)

      loss_likelihood,last_grad,last_grad2=BAOAB_step2(net,net2,hc,hper2c,batch_it_list,last_grad,last_grad2)

      sum_loss=sum_loss+loss_likelihood

    print('Epoch [%d/%d], Step [%d/%d]' %(epoch+1, num_epochs, i+1, no_batches))
    correct = 0
    total = 0  

    with torch.no_grad():
      for testit in range(test_no_batches):
        imagest=test_images_list[testit]
        labelst=test_labels_list[testit]
        actual_batch_size=len(imagest)
        test_labels_arr[(testit*batch_size):(testit*batch_size+actual_batch_size)]=labelst.detach().cpu()
        outputt = net(imagest).detach()#.reshape(actual_batch_size).detach()
        outputt2 = net2(imagest).detach()#.reshape(actual_batch_size).detach()
        _, predictedt = torch.max(outputt,1)
        #_, predictedt2 = torch.max(outputt2,1)
        correct += (predictedt == labelst).sum()
        total += labelst.size(0)

        test_prob_arr[(testit*batch_size):(testit*batch_size+actual_batch_size),0:num_classes,epoch]=torch.softmax(outputt,dim=1)
        test_prob_arr2[(testit*batch_size):(testit*batch_size+actual_batch_size),0:num_classes,epoch]=torch.softmax(outputt2,dim=1)
    
    #net.train()       
    print('Test accuracy of the model: %.3f %%' %((100*correct)/(total+1)))
    print('Epoch [%d], Average Loss: %0.4f' %(epoch+1, sum_loss/no_batches))
  return(test_labels_arr,test_prob_arr,test_prob_arr2)


def SMS_BAOAB2(num_epochs,h,gam):
  net=copy.deepcopy(net_star)
  net2=copy.deepcopy(net_star)  
  with torch.no_grad():
    hc=BAOAB_hconst(h,gam)
    hper2c=BAOAB_hconst(h/2,gam)      
    test_labels_arr=torch.zeros(test_size)
    test_prob_arr=torch.zeros([test_size,num_classes,num_epochs])
    test_prob_arr2=torch.zeros([test_size,num_classes,num_epochs])  
  #Initialise velocities
    for par,par2 in zip(net.parameters(), net2.parameters()):
      par.v = torch.randn_like(par,device=device).detach()
      par2.v=copy.deepcopy(par.v).detach()
  
  for epoch in range(num_epochs):
    sum_loss=0
    if(epoch%2==0):
      rperm=random.permutation(list(range(no_batches)))      
    rperm2=random.permutation(list(range(no_batches)))
    for i in range(no_batches):
      b=i        
      it=epoch*batch_size+b
      ind=ind_create(2*it)
      ind2=ind_create(2*it+1)            
      indc=ind_create(it)          
      batch_it_list=[rperm2[ind], rperm2[ind2], rperm[indc]]

      if(epoch==0 and i==0):
        last_grad,_=svrg_grad(net,ind)
        last_grad2,_=svrg_grad(net2,indc)

      loss_likelihood,last_grad,last_grad2=BAOAB_step2(net,net2,hc,hper2c,batch_it_list,last_grad,last_grad2)

      sum_loss=sum_loss+loss_likelihood

    print('Epoch [%d/%d], Step [%d/%d]' %(epoch+1, num_epochs, i+1, no_batches))
    correct = 0
    total = 0  

    with torch.no_grad():
      for testit in range(test_no_batches):
        imagest=test_images_list[testit]
        labelst=test_labels_list[testit]
        actual_batch_size=len(imagest)
        test_labels_arr[(testit*batch_size):(testit*batch_size+actual_batch_size)]=labelst.detach().cpu()
        outputt = net(imagest).detach()#.reshape(actual_batch_size).detach()
        outputt2 = net2(imagest).detach()#.reshape(actual_batch_size).detach()
        _, predictedt = torch.max(outputt,1)
        #_, predictedt2 = torch.max(outputt2,1)
        correct += (predictedt == labelst).sum()
        total += labelst.size(0)

        test_prob_arr[(testit*batch_size):(testit*batch_size+actual_batch_size),0:num_classes,epoch]=torch.softmax(outputt,dim=1)
        test_prob_arr2[(testit*batch_size):(testit*batch_size+actual_batch_size),0:num_classes,epoch]=torch.softmax(outputt2,dim=1)
    
    #net.train()       
    print('Test accuracy of the model: %.3f %%' %((100*correct)/(total+1)))
    print('Epoch [%d], Average Loss: %0.4f' %(epoch+1, sum_loss/no_batches))
  return(test_labels_arr,test_prob_arr,test_prob_arr2)

net_star, net_star_grad_list, net_star_full_grad, len_params, list_no_bias=find_MAP(40)


In [ ]:
rat=0.5
num_epochs=int(400*rat)
h= torch.tensor(1e-3)/rat
test_labels_arr,test_prob_arr,test_prob_arr2=SG_BAOAB2(num_epochs,h,gam)
filepath="output_fashion_multinomial_SG_BAOAB_h-1.pickle"
with open(filepath,"wb") as file:
  pickle.dump([test_labels_arr,test_prob_arr, test_prob_arr2],file)

test_prob=torch.Tensor(test_prob_arr[:,:,int(40*rat):num_epochs]).mean(-1).reshape(test_size,num_classes)
test_prob2=torch.Tensor(test_prob_arr2[:,:,int(40*rat):num_epochs]).mean(-1).reshape(test_size,num_classes)

test_prob_correct_label=torch.Tensor([test_prob[it, test_labels_arr[it].int()] for it in range(test_size)])
test_prob_correct_label2=torch.Tensor([test_prob2[it, test_labels_arr[it].int()] for it in range(test_size)])
print("Average bias in probability of true category", (test_prob_correct_label-test_prob_correct_label2).abs().mean())

In [ ]:
rat=1
num_epochs=400*rat
h= torch.tensor(1e-3)/rat
test_labels_arr,test_prob_arr,test_prob_arr2=SG_BAOAB2(num_epochs,h,gam)
filepath="output_fashion_multinomial_SG_BAOAB_h0.pickle"
with open(filepath,"wb") as file:
  pickle.dump([test_labels_arr,test_prob_arr, test_prob_arr2],file)

test_prob=torch.Tensor(test_prob_arr[:,:,40*rat:num_epochs]).mean(-1).reshape(test_size,num_classes)
test_prob2=torch.Tensor(test_prob_arr2[:,:,40*rat:num_epochs]).mean(-1).reshape(test_size,num_classes)

test_prob_correct_label=torch.Tensor([test_prob[it, test_labels_arr[it].int()] for it in range(test_size)])
test_prob_correct_label2=torch.Tensor([test_prob2[it, test_labels_arr[it].int()] for it in range(test_size)])
print("Average bias in probability of true category", (test_prob_correct_label-test_prob_correct_label2).abs().mean())


In [ ]:
rat=2
num_epochs=400*rat
h=torch.tensor(1e-3)/rat
test_labels_arr,test_prob_arr,test_prob_arr2=SG_BAOAB2(num_epochs,h,gam)
filepath="output_fashion_multinomial_SG_BAOAB_h1.pickle"
with open(filepath,"wb") as file:
  pickle.dump([test_labels_arr,test_prob_arr, test_prob_arr2],file)

test_prob=torch.Tensor(test_prob_arr[:,:,40*rat:num_epochs]).mean(-1).reshape(test_size,num_classes)
test_prob2=torch.Tensor(test_prob_arr2[:,:,40*rat:num_epochs]).mean(-1).reshape(test_size,num_classes)

test_prob_correct_label=torch.Tensor([test_prob[it, test_labels_arr[it].int()] for it in range(test_size)])
test_prob_correct_label2=torch.Tensor([test_prob2[it, test_labels_arr[it].int()] for it in range(test_size)])
print("Average bias in probability of true category", (test_prob_correct_label-test_prob_correct_label2).abs().mean())

In [ ]:
rat=4
num_epochs=400*rat
h=torch.tensor(1e-3)/rat
test_labels_arr,test_prob_arr,test_prob_arr2=SG_BAOAB2(num_epochs,h,gam)
filepath="output_fashion_multinomial_SG_BAOAB_h2.pickle"
with open(filepath,"wb") as file:
  pickle.dump([test_labels_arr,test_prob_arr, test_prob_arr2],file)

test_prob=torch.Tensor(test_prob_arr[:,:,40*rat:num_epochs]).mean(-1).reshape(test_size,num_classes)
test_prob2=torch.Tensor(test_prob_arr2[:,:,40*rat:num_epochs]).mean(-1).reshape(test_size,num_classes)

test_prob_correct_label=torch.Tensor([test_prob[it, test_labels_arr[it].int()] for it in range(test_size)])
test_prob_correct_label2=torch.Tensor([test_prob2[it, test_labels_arr[it].int()] for it in range(test_size)])
print("Average bias in probability of true category", (test_prob_correct_label-test_prob_correct_label2).abs().mean())

In [ ]:
rat=8
num_epochs=400*rat
h=torch.tensor(1e-3)/rat
test_labels_arr,test_prob_arr,test_prob_arr2=SG_BAOAB2(num_epochs,h,gam)
filepath="output_fashion_multinomial_SG_BAOAB_h3.pickle"
with open(filepath,"wb") as file:
  pickle.dump([test_labels_arr,test_prob_arr, test_prob_arr2],file)

test_prob=torch.Tensor(test_prob_arr[:,:,40*rat:num_epochs]).mean(-1).reshape(test_size,num_classes)
test_prob2=torch.Tensor(test_prob_arr2[:,:,40*rat:num_epochs]).mean(-1).reshape(test_size,num_classes)

test_prob_correct_label=torch.Tensor([test_prob[it, test_labels_arr[it].int()] for it in range(test_size)])
test_prob_correct_label2=torch.Tensor([test_prob2[it, test_labels_arr[it].int()] for it in range(test_size)])
print("Average bias in probability of true category", (test_prob_correct_label-test_prob_correct_label2).abs().mean())

In [ ]:
rat=16
num_epochs=400*rat
h=torch.tensor(1e-3)/rat
test_labels_arr,test_prob_arr,test_prob_arr2=SG_BAOAB2(num_epochs,h,gam)
filepath="output_fashion_multinomial_SG_BAOAB_h4.pickle"
with open(filepath,"wb") as file:
  pickle.dump([test_labels_arr,test_prob_arr, test_prob_arr2],file)

test_prob=torch.Tensor(test_prob_arr[:,:,40*rat:num_epochs]).mean(-1).reshape(test_size,num_classes)
test_prob2=torch.Tensor(test_prob_arr2[:,:,40*rat:num_epochs]).mean(-1).reshape(test_size,num_classes)

test_prob_correct_label=torch.Tensor([test_prob[it, test_labels_arr[it].int()] for it in range(test_size)])
test_prob_correct_label2=torch.Tensor([test_prob2[it, test_labels_arr[it].int()] for it in range(test_size)])
print("Average bias in probability of true category", (test_prob_correct_label-test_prob_correct_label2).abs().mean())

In [ ]:
rat=32
num_epochs=400*rat
h=torch.tensor(1e-3)/rat
test_labels_arr,test_prob_arr,test_prob_arr2=SG_BAOAB2(num_epochs,h,gam)
filepath="output_fashion_multinomial_SG_BAOAB_h5.pickle"
with open(filepath,"wb") as file:
  pickle.dump([test_labels_arr,test_prob_arr, test_prob_arr2],file)

test_prob=torch.Tensor(test_prob_arr[:,:,40*rat:num_epochs]).mean(-1).reshape(test_size,num_classes)
test_prob2=torch.Tensor(test_prob_arr2[:,:,40*rat:num_epochs]).mean(-1).reshape(test_size,num_classes)

test_prob_correct_label=torch.Tensor([test_prob[it, test_labels_arr[it].int()] for it in range(test_size)])
test_prob_correct_label2=torch.Tensor([test_prob2[it, test_labels_arr[it].int()] for it in range(test_size)])
print("Average bias in probability of true category", (test_prob_correct_label-test_prob_correct_label2).abs().mean())

In [ ]:
rat=0.5
num_epochs=int(400*rat)
h= torch.tensor(1e-3)/rat
test_labels_arr,test_prob_arr,test_prob_arr2=SMS_BAOAB2(num_epochs,h,gam)
filepath="output_fashion_multinomial_SMS_BAOAB_h-1.pickle"
with open(filepath,"wb") as file:
  pickle.dump([test_labels_arr,test_prob_arr, test_prob_arr2],file)

test_prob=torch.Tensor(test_prob_arr[:,:,int(40*rat):num_epochs]).mean(-1).reshape(test_size,num_classes)
test_prob2=torch.Tensor(test_prob_arr2[:,:,int(40*rat):num_epochs]).mean(-1).reshape(test_size,num_classes)

test_prob_correct_label=torch.Tensor([test_prob[it, test_labels_arr[it].int()] for it in range(test_size)])
test_prob_correct_label2=torch.Tensor([test_prob2[it, test_labels_arr[it].int()] for it in range(test_size)])
print("Average bias in probability of true category", (test_prob_correct_label-test_prob_correct_label2).abs().mean())

In [ ]:
rat=1
num_epochs=400*rat
h= torch.tensor(1e-3)/rat
test_labels_arr,test_prob_arr,test_prob_arr2=SMS_BAOAB2(num_epochs,h,gam)
filepath="output_fashion_multinomial_SMS_BAOAB_h0.pickle"
with open(filepath,"wb") as file:
  pickle.dump([test_labels_arr,test_prob_arr, test_prob_arr2],file)

test_prob=torch.Tensor(test_prob_arr[:,:,40*rat:num_epochs]).mean(-1).reshape(test_size,num_classes)
test_prob2=torch.Tensor(test_prob_arr2[:,:,40*rat:num_epochs]).mean(-1).reshape(test_size,num_classes)

test_prob_correct_label=torch.Tensor([test_prob[it, test_labels_arr[it].int()] for it in range(test_size)])
test_prob_correct_label2=torch.Tensor([test_prob2[it, test_labels_arr[it].int()] for it in range(test_size)])
print("Average bias in probability of true category", (test_prob_correct_label-test_prob_correct_label2).abs().mean())

In [ ]:
rat=2
num_epochs=400*rat
h= torch.tensor(1e-3)/rat
test_labels_arr,test_prob_arr,test_prob_arr2=SMS_BAOAB2(num_epochs,h,gam)
filepath="output_fashion_multinomial_SMS_BAOAB_h1.pickle"
with open(filepath,"wb") as file:
  pickle.dump([test_labels_arr,test_prob_arr, test_prob_arr2],file)

test_prob=torch.Tensor(test_prob_arr[:,:,40*rat:num_epochs]).mean(-1).reshape(test_size,num_classes)
test_prob2=torch.Tensor(test_prob_arr2[:,:,40*rat:num_epochs]).mean(-1).reshape(test_size,num_classes)

test_prob_correct_label=torch.Tensor([test_prob[it, test_labels_arr[it].int()] for it in range(test_size)])
test_prob_correct_label2=torch.Tensor([test_prob2[it, test_labels_arr[it].int()] for it in range(test_size)])
print("Average bias in probability of true category", (test_prob_correct_label-test_prob_correct_label2).abs().mean())

In [ ]:
rat=4
num_epochs=400*rat
h= torch.tensor(1e-3)/rat
test_labels_arr,test_prob_arr,test_prob_arr2=SMS_BAOAB2(num_epochs,h,gam)
filepath="output_fashion_multinomial_SMS_BAOAB_h2.pickle"
with open(filepath,"wb") as file:
  pickle.dump([test_labels_arr,test_prob_arr, test_prob_arr2],file)

test_prob=torch.Tensor(test_prob_arr[:,:,40*rat:num_epochs]).mean(-1).reshape(test_size,num_classes)
test_prob2=torch.Tensor(test_prob_arr2[:,:,40*rat:num_epochs]).mean(-1).reshape(test_size,num_classes)

test_prob_correct_label=torch.Tensor([test_prob[it, test_labels_arr[it].int()] for it in range(test_size)])
test_prob_correct_label2=torch.Tensor([test_prob2[it, test_labels_arr[it].int()] for it in range(test_size)])
print("Average bias in probability of true category", (test_prob_correct_label-test_prob_correct_label2).abs().mean())

In [ ]:
rat=8
num_epochs=400*rat
h= torch.tensor(1e-3)/rat
test_labels_arr,test_prob_arr,test_prob_arr2=SMS_BAOAB2(num_epochs,h,gam)
filepath="output_fashion_multinomial_SMS_BAOAB_h3.pickle"
with open(filepath,"wb") as file:
  pickle.dump([test_labels_arr,test_prob_arr, test_prob_arr2],file)

test_prob=torch.Tensor(test_prob_arr[:,:,40*rat:num_epochs]).mean(-1).reshape(test_size,num_classes)
test_prob2=torch.Tensor(test_prob_arr2[:,:,40*rat:num_epochs]).mean(-1).reshape(test_size,num_classes)

test_prob_correct_label=torch.Tensor([test_prob[it, test_labels_arr[it].int()] for it in range(test_size)])
test_prob_correct_label2=torch.Tensor([test_prob2[it, test_labels_arr[it].int()] for it in range(test_size)])
print("Average bias in probability of true category", (test_prob_correct_label-test_prob_correct_label2).abs().mean())

In [ ]:
rat=16
num_epochs=400*rat
h= torch.tensor(1e-3)/rat
test_labels_arr,test_prob_arr,test_prob_arr2=SMS_BAOAB2(num_epochs,h,gam)
filepath="output_fashion_multinomial_SMS_BAOAB_h4.pickle"
with open(filepath,"wb") as file:
  pickle.dump([test_labels_arr,test_prob_arr, test_prob_arr2],file)

test_prob=torch.Tensor(test_prob_arr[:,:,40*rat:num_epochs]).mean(-1).reshape(test_size,num_classes)
test_prob2=torch.Tensor(test_prob_arr2[:,:,40*rat:num_epochs]).mean(-1).reshape(test_size,num_classes)

test_prob_correct_label=torch.Tensor([test_prob[it, test_labels_arr[it].int()] for it in range(test_size)])
test_prob_correct_label2=torch.Tensor([test_prob2[it, test_labels_arr[it].int()] for it in range(test_size)])
print("Average bias in probability of true category", (test_prob_correct_label-test_prob_correct_label2).abs().mean())

In [ ]:
rat=32
num_epochs=400*rat
h= torch.tensor(1e-3)/rat
test_labels_arr,test_prob_arr,test_prob_arr2=SMS_BAOAB2(num_epochs,h,gam)
filepath="output_fashion_multinomial_SMS_BAOAB_h5.pickle"
with open(filepath,"wb") as file:
  pickle.dump([test_labels_arr,test_prob_arr, test_prob_arr2],file)

test_prob=torch.Tensor(test_prob_arr[:,:,40*rat:num_epochs]).mean(-1).reshape(test_size,num_classes)
test_prob2=torch.Tensor(test_prob_arr2[:,:,40*rat:num_epochs]).mean(-1).reshape(test_size,num_classes)

test_prob_correct_label=torch.Tensor([test_prob[it, test_labels_arr[it].int()] for it in range(test_size)])
test_prob_correct_label2=torch.Tensor([test_prob2[it, test_labels_arr[it].int()] for it in range(test_size)])
print("Average bias in probability of true category", (test_prob_correct_label-test_prob_correct_label2).abs().mean())

In [ ]:
rat=0.5
num_epochs=int(400*rat)
h= torch.tensor(1e-3)/rat
test_labels_arr,test_prob_arr,test_prob_arr2=SG_UBU2(num_epochs,h,gam)
filepath="output_fashion_multinomial_SG_UBU_h-1.pickle"
with open(filepath,"wb") as file:
  pickle.dump([test_labels_arr,test_prob_arr, test_prob_arr2],file)

test_prob=torch.Tensor(test_prob_arr[:,:,int(40*rat):num_epochs]).mean(-1).reshape(test_size,num_classes)
test_prob2=torch.Tensor(test_prob_arr2[:,:,int(40*rat):num_epochs]).mean(-1).reshape(test_size,num_classes)

test_prob_correct_label=torch.Tensor([test_prob[it, test_labels_arr[it].int()] for it in range(test_size)])
test_prob_correct_label2=torch.Tensor([test_prob2[it, test_labels_arr[it].int()] for it in range(test_size)])
print("Average bias in probability of true category", (test_prob_correct_label-test_prob_correct_label2).abs().mean())

In [ ]:
rat=1
num_epochs=400*rat
h= torch.tensor(1e-3)/rat
test_labels_arr,test_prob_arr,test_prob_arr2=SG_UBU2(num_epochs,h,gam)
filepath="output_fashion_multinomial_SG_UBU_h0.pickle"
with open(filepath,"wb") as file:
  pickle.dump([test_labels_arr,test_prob_arr, test_prob_arr2],file)

test_prob=torch.Tensor(test_prob_arr[:,:,40*rat:num_epochs]).mean(-1).reshape(test_size,num_classes)
test_prob2=torch.Tensor(test_prob_arr2[:,:,40*rat:num_epochs]).mean(-1).reshape(test_size,num_classes)

test_prob_correct_label=torch.Tensor([test_prob[it, test_labels_arr[it].int()] for it in range(test_size)])
test_prob_correct_label2=torch.Tensor([test_prob2[it, test_labels_arr[it].int()] for it in range(test_size)])
print("Average bias in probability of true category", (test_prob_correct_label-test_prob_correct_label2).abs().mean())

In [ ]:
rat=2
num_epochs=400*rat
h= torch.tensor(1e-3)/rat
test_labels_arr,test_prob_arr,test_prob_arr2=SG_UBU2(num_epochs,h,gam)
filepath="output_fashion_multinomial_SG_UBU_h1.pickle"
with open(filepath,"wb") as file:
  pickle.dump([test_labels_arr,test_prob_arr, test_prob_arr2],file)

test_prob=torch.Tensor(test_prob_arr[:,:,40*rat:num_epochs]).mean(-1).reshape(test_size,num_classes)
test_prob2=torch.Tensor(test_prob_arr2[:,:,40*rat:num_epochs]).mean(-1).reshape(test_size,num_classes)

test_prob_correct_label=torch.Tensor([test_prob[it, test_labels_arr[it].int()] for it in range(test_size)])
test_prob_correct_label2=torch.Tensor([test_prob2[it, test_labels_arr[it].int()] for it in range(test_size)])
print("Average bias in probability of true category", (test_prob_correct_label-test_prob_correct_label2).abs().mean())

In [ ]:
rat=4
num_epochs=400*rat
h= torch.tensor(1e-3)/rat
test_labels_arr,test_prob_arr,test_prob_arr2=SG_UBU2(num_epochs,h,gam)
filepath="output_fashion_multinomial_SG_UBU_h2.pickle"
with open(filepath,"wb") as file:
  pickle.dump([test_labels_arr,test_prob_arr, test_prob_arr2],file)

test_prob=torch.Tensor(test_prob_arr[:,:,40*rat:num_epochs]).mean(-1).reshape(test_size,num_classes)
test_prob2=torch.Tensor(test_prob_arr2[:,:,40*rat:num_epochs]).mean(-1).reshape(test_size,num_classes)

test_prob_correct_label=torch.Tensor([test_prob[it, test_labels_arr[it].int()] for it in range(test_size)])
test_prob_correct_label2=torch.Tensor([test_prob2[it, test_labels_arr[it].int()] for it in range(test_size)])
print("Average bias in probability of true category", (test_prob_correct_label-test_prob_correct_label2).abs().mean())

In [ ]:
rat=8
num_epochs=400*rat
h= torch.tensor(1e-3)/rat
test_labels_arr,test_prob_arr,test_prob_arr2=SG_UBU2(num_epochs,h,gam)
filepath="output_fashion_multinomial_SG_UBU_h3.pickle"
with open(filepath,"wb") as file:
  pickle.dump([test_labels_arr,test_prob_arr, test_prob_arr2],file)

test_prob=torch.Tensor(test_prob_arr[:,:,40*rat:num_epochs]).mean(-1).reshape(test_size,num_classes)
test_prob2=torch.Tensor(test_prob_arr2[:,:,40*rat:num_epochs]).mean(-1).reshape(test_size,num_classes)

test_prob_correct_label=torch.Tensor([test_prob[it, test_labels_arr[it].int()] for it in range(test_size)])
test_prob_correct_label2=torch.Tensor([test_prob2[it, test_labels_arr[it].int()] for it in range(test_size)])
print("Average bias in probability of true category", (test_prob_correct_label-test_prob_correct_label2).abs().mean())

In [ ]:
rat=16
num_epochs=400*rat
h= torch.tensor(1e-3)/rat
test_labels_arr,test_prob_arr,test_prob_arr2=SG_UBU2(num_epochs,h,gam)
filepath="output_fashion_multinomial_SG_UBU_h4.pickle"
with open(filepath,"wb") as file:
  pickle.dump([test_labels_arr,test_prob_arr, test_prob_arr2],file)

test_prob=torch.Tensor(test_prob_arr[:,:,40*rat:num_epochs]).mean(-1).reshape(test_size,num_classes)
test_prob2=torch.Tensor(test_prob_arr2[:,:,40*rat:num_epochs]).mean(-1).reshape(test_size,num_classes)

test_prob_correct_label=torch.Tensor([test_prob[it, test_labels_arr[it].int()] for it in range(test_size)])
test_prob_correct_label2=torch.Tensor([test_prob2[it, test_labels_arr[it].int()] for it in range(test_size)])
print("Average bias in probability of true category", (test_prob_correct_label-test_prob_correct_label2).abs().mean())

In [ ]:
rat=32
num_epochs=400*rat
h= torch.tensor(1e-3)/rat
test_labels_arr,test_prob_arr,test_prob_arr2=SG_UBU2(num_epochs,h,gam)
filepath="output_fashion_multinomial_SG_UBU_h5.pickle"
with open(filepath,"wb") as file:
  pickle.dump([test_labels_arr,test_prob_arr, test_prob_arr2],file)

test_prob=torch.Tensor(test_prob_arr[:,:,40*rat:num_epochs]).mean(-1).reshape(test_size,num_classes)
test_prob2=torch.Tensor(test_prob_arr2[:,:,40*rat:num_epochs]).mean(-1).reshape(test_size,num_classes)

test_prob_correct_label=torch.Tensor([test_prob[it, test_labels_arr[it].int()] for it in range(test_size)])
test_prob_correct_label2=torch.Tensor([test_prob2[it, test_labels_arr[it].int()] for it in range(test_size)])
print("Average bias in probability of true category", (test_prob_correct_label-test_prob_correct_label2).abs().mean())

In [ ]:
rat=0.5
num_epochs=int(400*rat)
h= torch.tensor(1e-3)/rat
test_labels_arr,test_prob_arr,test_prob_arr2=SMS_UBU2(num_epochs,h,gam)
filepath="output_fashion_multinomial_SMS_UBU_h-1.pickle"
with open(filepath,"wb") as file:
  pickle.dump([test_labels_arr,test_prob_arr, test_prob_arr2],file)

test_prob=torch.Tensor(test_prob_arr[:,:,int(40*rat):num_epochs]).mean(-1).reshape(test_size,num_classes)
test_prob2=torch.Tensor(test_prob_arr2[:,:,int(40*rat):num_epochs]).mean(-1).reshape(test_size,num_classes)

test_prob_correct_label=torch.Tensor([test_prob[it, test_labels_arr[it].int()] for it in range(test_size)])
test_prob_correct_label2=torch.Tensor([test_prob2[it, test_labels_arr[it].int()] for it in range(test_size)])
print("Average bias in probability of true category", (test_prob_correct_label-test_prob_correct_label2).abs().mean())

In [ ]:
rat=1
num_epochs=400*rat
h= torch.tensor(1e-3)/rat
test_labels_arr,test_prob_arr,test_prob_arr2=SMS_UBU2(num_epochs,h,gam)
filepath="output_fashion_multinomial_SMS_UBU_h0.pickle"
with open(filepath,"wb") as file:
  pickle.dump([test_labels_arr,test_prob_arr, test_prob_arr2],file)

test_prob=torch.Tensor(test_prob_arr[:,:,40*rat:num_epochs]).mean(-1).reshape(test_size,num_classes)
test_prob2=torch.Tensor(test_prob_arr2[:,:,40*rat:num_epochs]).mean(-1).reshape(test_size,num_classes)

test_prob_correct_label=torch.Tensor([test_prob[it, test_labels_arr[it].int()] for it in range(test_size)])
test_prob_correct_label2=torch.Tensor([test_prob2[it, test_labels_arr[it].int()] for it in range(test_size)])
print("Average bias in probability of true category", (test_prob_correct_label-test_prob_correct_label2).abs().mean())

In [ ]:
rat=2
num_epochs=400*rat
h= torch.tensor(1e-3)/rat
test_labels_arr,test_prob_arr,test_prob_arr2=SMS_UBU2(num_epochs,h,gam)
filepath="output_fashion_multinomial_SMS_UBU_h1.pickle"
with open(filepath,"wb") as file:
  pickle.dump([test_labels_arr,test_prob_arr, test_prob_arr2],file)

test_prob=torch.Tensor(test_prob_arr[:,:,40*rat:num_epochs]).mean(-1).reshape(test_size,num_classes)
test_prob2=torch.Tensor(test_prob_arr2[:,:,40*rat:num_epochs]).mean(-1).reshape(test_size,num_classes)

test_prob_correct_label=torch.Tensor([test_prob[it, test_labels_arr[it].int()] for it in range(test_size)])
test_prob_correct_label2=torch.Tensor([test_prob2[it, test_labels_arr[it].int()] for it in range(test_size)])
print("Average bias in probability of true category", (test_prob_correct_label-test_prob_correct_label2).abs().mean())

In [ ]:
rat=4
num_epochs=400*rat
h= torch.tensor(1e-3)/rat
test_labels_arr,test_prob_arr,test_prob_arr2=SMS_UBU2(num_epochs,h,gam)
filepath="output_fashion_multinomial_SMS_UBU_h2.pickle"
with open(filepath,"wb") as file:
  pickle.dump([test_labels_arr,test_prob_arr, test_prob_arr2],file)

test_prob=torch.Tensor(test_prob_arr[:,:,40*rat:num_epochs]).mean(-1).reshape(test_size,num_classes)
test_prob2=torch.Tensor(test_prob_arr2[:,:,40*rat:num_epochs]).mean(-1).reshape(test_size,num_classes)

test_prob_correct_label=torch.Tensor([test_prob[it, test_labels_arr[it].int()] for it in range(test_size)])
test_prob_correct_label2=torch.Tensor([test_prob2[it, test_labels_arr[it].int()] for it in range(test_size)])
print("Average bias in probability of true category", (test_prob_correct_label-test_prob_correct_label2).abs().mean())

In [ ]:
rat=8
num_epochs=400*rat
h= torch.tensor(1e-3)/rat
test_labels_arr,test_prob_arr,test_prob_arr2=SMS_UBU2(num_epochs,h,gam)
filepath="output_fashion_multinomial_SMS_UBU_h3.pickle"
with open(filepath,"wb") as file:
  pickle.dump([test_labels_arr,test_prob_arr, test_prob_arr2],file)

test_prob=torch.Tensor(test_prob_arr[:,:,40*rat:num_epochs]).mean(-1).reshape(test_size,num_classes)
test_prob2=torch.Tensor(test_prob_arr2[:,:,40*rat:num_epochs]).mean(-1).reshape(test_size,num_classes)

test_prob_correct_label=torch.Tensor([test_prob[it, test_labels_arr[it].int()] for it in range(test_size)])
test_prob_correct_label2=torch.Tensor([test_prob2[it, test_labels_arr[it].int()] for it in range(test_size)])
print("Average bias in probability of true category", (test_prob_correct_label-test_prob_correct_label2).abs().mean())

In [ ]:
rat=16
num_epochs=800*rat
h= torch.tensor(1e-3)/rat
test_labels_arr,test_prob_arr,test_prob_arr2=SMS_UBU2(num_epochs,h,gam)
filepath="output_fashion_multinomial_SMS_UBU_h4l.pickle"
with open(filepath,"wb") as file:
  pickle.dump([test_labels_arr,test_prob_arr, test_prob_arr2],file)

test_prob=torch.Tensor(test_prob_arr[:,:,10*rat:num_epochs]).mean(-1).reshape(test_size,num_classes)
test_prob2=torch.Tensor(test_prob_arr2[:,:,10*rat:num_epochs]).mean(-1).reshape(test_size,num_classes)

test_prob_correct_label=torch.Tensor([test_prob[it, test_labels_arr[it].int()] for it in range(test_size)])
test_prob_correct_label2=torch.Tensor([test_prob2[it, test_labels_arr[it].int()] for it in range(test_size)])
print("Average bias in probability of true category", (test_prob_correct_label-test_prob_correct_label2).abs().mean())

In [ ]:
rat=16
num_epochs=400*rat
h= torch.tensor(1e-3)/rat
test_labels_arr,test_prob_arr,test_prob_arr2=SMS_UBU2(num_epochs,h,gam)
filepath="output_fashion_multinomial_SMS_UBU_h4.pickle"
with open(filepath,"wb") as file:
  pickle.dump([test_labels_arr,test_prob_arr, test_prob_arr2],file)

test_prob=torch.Tensor(test_prob_arr[:,:,40*rat:num_epochs]).mean(-1).reshape(test_size,num_classes)
test_prob2=torch.Tensor(test_prob_arr2[:,:,40*rat:num_epochs]).mean(-1).reshape(test_size,num_classes)

test_prob_correct_label=torch.Tensor([test_prob[it, test_labels_arr[it].int()] for it in range(test_size)])
test_prob_correct_label2=torch.Tensor([test_prob2[it, test_labels_arr[it].int()] for it in range(test_size)])
print("Average bias in probability of true category", (test_prob_correct_label-test_prob_correct_label2).abs().mean())

In [ ]:
rat=32
num_epochs=400*rat
h= torch.tensor(1e-3)/rat
test_labels_arr,test_prob_arr,test_prob_arr2=SMS_UBU2(num_epochs,h,gam)
filepath="output_fashion_multinomial_SMS_UBU_h5.pickle"
with open(filepath,"wb") as file:
  pickle.dump([test_labels_arr,test_prob_arr, test_prob_arr2],file)

test_prob=torch.Tensor(test_prob_arr[:,:,40*rat:num_epochs]).mean(-1).reshape(test_size,num_classes)
test_prob2=torch.Tensor(test_prob_arr2[:,:,40*rat:num_epochs]).mean(-1).reshape(test_size,num_classes)

test_prob_correct_label=torch.Tensor([test_prob[it, test_labels_arr[it].int()] for it in range(test_size)])
test_prob_correct_label2=torch.Tensor([test_prob2[it, test_labels_arr[it].int()] for it in range(test_size)])
print("Average bias in probability of true category", (test_prob_correct_label-test_prob_correct_label2).abs().mean())

In [ ]:
rat=0.5
num_epochs=int(400*rat)
h= torch.tensor(1e-3)/rat
test_labels_arr,test_prob_arr,test_prob_arr2=SG_EM2(num_epochs,h,gam)
filepath="output_fashion_multinomial_SG_EM_h-1.pickle"
with open(filepath,"wb") as file:
  pickle.dump([test_labels_arr,test_prob_arr, test_prob_arr2],file)

test_prob=torch.Tensor(test_prob_arr[:,:,int(40*rat):num_epochs]).mean(-1).reshape(test_size,num_classes)
test_prob2=torch.Tensor(test_prob_arr2[:,:,int(40*rat):num_epochs]).mean(-1).reshape(test_size,num_classes)

test_prob_correct_label=torch.Tensor([test_prob[it, test_labels_arr[it].int()] for it in range(test_size)])
test_prob_correct_label2=torch.Tensor([test_prob2[it, test_labels_arr[it].int()] for it in range(test_size)])
print("Average bias in probability of true category", (test_prob_correct_label-test_prob_correct_label2).abs().mean())

In [ ]:
rat=1
num_epochs=400*rat
h= torch.tensor(1e-3)/rat
test_labels_arr,test_prob_arr,test_prob_arr2=SG_EM2(num_epochs,h,gam)
filepath="output_fashion_multinomial_SG_EM_h0.pickle"
with open(filepath,"wb") as file:
  pickle.dump([test_labels_arr,test_prob_arr, test_prob_arr2],file)

test_prob=torch.Tensor(test_prob_arr[:,:,40*rat:num_epochs]).mean(-1).reshape(test_size,num_classes)
test_prob2=torch.Tensor(test_prob_arr2[:,:,40*rat:num_epochs]).mean(-1).reshape(test_size,num_classes)

test_prob_correct_label=torch.Tensor([test_prob[it, test_labels_arr[it].int()] for it in range(test_size)])
test_prob_correct_label2=torch.Tensor([test_prob2[it, test_labels_arr[it].int()] for it in range(test_size)])
print("Average bias in probability of true category", (test_prob_correct_label-test_prob_correct_label2).abs().mean())

In [ ]:
rat=2
num_epochs=400*rat
h= torch.tensor(1e-3)/rat
test_labels_arr,test_prob_arr,test_prob_arr2=SG_EM2(num_epochs,h,gam)
filepath="output_fashion_multinomial_SG_EM_h1.pickle"
with open(filepath,"wb") as file:
  pickle.dump([test_labels_arr,test_prob_arr, test_prob_arr2],file)

test_prob=torch.Tensor(test_prob_arr[:,:,40*rat:num_epochs]).mean(-1).reshape(test_size,num_classes)
test_prob2=torch.Tensor(test_prob_arr2[:,:,40*rat:num_epochs]).mean(-1).reshape(test_size,num_classes)

test_prob_correct_label=torch.Tensor([test_prob[it, test_labels_arr[it].int()] for it in range(test_size)])
test_prob_correct_label2=torch.Tensor([test_prob2[it, test_labels_arr[it].int()] for it in range(test_size)])
print("Average bias in probability of true category", (test_prob_correct_label-test_prob_correct_label2).abs().mean())

In [ ]:
rat=4
num_epochs=400*rat
h= torch.tensor(1e-3)/rat
test_labels_arr,test_prob_arr,test_prob_arr2=SG_EM2(num_epochs,h,gam)
filepath="output_fashion_multinomial_SG_EM_h2.pickle"
with open(filepath,"wb") as file:
  pickle.dump([test_labels_arr,test_prob_arr, test_prob_arr2],file)

test_prob=torch.Tensor(test_prob_arr[:,:,40*rat:num_epochs]).mean(-1).reshape(test_size,num_classes)
test_prob2=torch.Tensor(test_prob_arr2[:,:,40*rat:num_epochs]).mean(-1).reshape(test_size,num_classes)

test_prob_correct_label=torch.Tensor([test_prob[it, test_labels_arr[it].int()] for it in range(test_size)])
test_prob_correct_label2=torch.Tensor([test_prob2[it, test_labels_arr[it].int()] for it in range(test_size)])
print("Average bias in probability of true category", (test_prob_correct_label-test_prob_correct_label2).abs().mean())

In [ ]:
rat=8
num_epochs=400*rat
h= torch.tensor(1e-3)/rat
test_labels_arr,test_prob_arr,test_prob_arr2=SG_EM2(num_epochs,h,gam)
filepath="output_fashion_multinomial_SG_EM_h3.pickle"
with open(filepath,"wb") as file:
  pickle.dump([test_labels_arr,test_prob_arr, test_prob_arr2],file)

test_prob=torch.Tensor(test_prob_arr[:,:,40*rat:num_epochs]).mean(-1).reshape(test_size,num_classes)
test_prob2=torch.Tensor(test_prob_arr2[:,:,40*rat:num_epochs]).mean(-1).reshape(test_size,num_classes)

test_prob_correct_label=torch.Tensor([test_prob[it, test_labels_arr[it].int()] for it in range(test_size)])
test_prob_correct_label2=torch.Tensor([test_prob2[it, test_labels_arr[it].int()] for it in range(test_size)])
print("Average bias in probability of true category", (test_prob_correct_label-test_prob_correct_label2).abs().mean())

In [ ]:
rat=16
num_epochs=400*rat
h= torch.tensor(1e-3)/rat
test_labels_arr,test_prob_arr,test_prob_arr2=SG_EM2(num_epochs,h,gam)
filepath="output_fashion_multinomial_SG_EM_h4.pickle"
with open(filepath,"wb") as file:
  pickle.dump([test_labels_arr,test_prob_arr, test_prob_arr2],file)

test_prob=torch.Tensor(test_prob_arr[:,:,40*rat:num_epochs]).mean(-1).reshape(test_size,num_classes)
test_prob2=torch.Tensor(test_prob_arr2[:,:,40*rat:num_epochs]).mean(-1).reshape(test_size,num_classes)

test_prob_correct_label=torch.Tensor([test_prob[it, test_labels_arr[it].int()] for it in range(test_size)])
test_prob_correct_label2=torch.Tensor([test_prob2[it, test_labels_arr[it].int()] for it in range(test_size)])
print("Average bias in probability of true category", (test_prob_correct_label-test_prob_correct_label2).abs().mean())

In [ ]:
rat=32
num_epochs=400*rat
h= torch.tensor(1e-3)/rat
test_labels_arr,test_prob_arr,test_prob_arr2=SG_EM2(num_epochs,h,gam)
filepath="output_fashion_multinomial_SG_EM_h5.pickle"
with open(filepath,"wb") as file:
  pickle.dump([test_labels_arr,test_prob_arr, test_prob_arr2],file)

test_prob=torch.Tensor(test_prob_arr[:,:,40*rat:num_epochs]).mean(-1).reshape(test_size,num_classes)
test_prob2=torch.Tensor(test_prob_arr2[:,:,40*rat:num_epochs]).mean(-1).reshape(test_size,num_classes)

test_prob_correct_label=torch.Tensor([test_prob[it, test_labels_arr[it].int()] for it in range(test_size)])
test_prob_correct_label2=torch.Tensor([test_prob2[it, test_labels_arr[it].int()] for it in range(test_size)])
print("Average bias in probability of true category", (test_prob_correct_label-test_prob_correct_label2).abs().mean())

In [ ]:
rat=64
num_epochs=800
h= torch.tensor(1e-3)/rat
test_labels_arr,test_prob_arr,test_prob_arr2=SG_EM2(num_epochs,h,gam)
# filepath="output_fashion_multinomial_SG_EM_h5.pickle"
# with open(filepath,"wb") as file:
#   pickle.dump([test_labels_arr,test_prob_arr, test_prob_arr2],file)

test_prob=torch.Tensor(test_prob_arr[:,:,40*rat:num_epochs]).mean(-1).reshape(test_size,num_classes)
test_prob2=torch.Tensor(test_prob_arr2[:,:,40*rat:num_epochs]).mean(-1).reshape(test_size,num_classes)

test_prob_correct_label=torch.Tensor([test_prob[it, test_labels_arr[it].int()] for it in range(test_size)])
test_prob_correct_label2=torch.Tensor([test_prob2[it, test_labels_arr[it].int()] for it in range(test_size)])
print("Average bias in probability of true category", (test_prob_correct_label-test_prob_correct_label2).abs().mean())

In [ ]:
def rps_single(probs, true_label,num_classes):
    outcome=torch.zeros(num_classes)
    outcome[true_label.int()]=1.0
    cum_probs = torch.cumsum(probs,0)
    cum_outcomes = torch.cumsum(outcome,0)
    
    sum_rps = 0
    for i in range(len(outcome)):         
        sum_rps+= (cum_probs[i] - cum_outcomes[i])**2
    
    return sum_rps/(num_classes-1)

def rps_calc(test_probs, true_labels,test_data_len,num_classes):
    rps_vec=torch.zeros(test_data_len)
    for it in range(test_data_len):
        rps_vec[it]=rps_single(test_probs[it,:].reshape(num_classes),true_labels[it],num_classes)
    return rps_vec

def nll_calc(test_probs, true_labels,test_data_len,num_classes):
    res=0
    for it in range(test_data_len):
        res-=torch.max(torch.tensor([torch.log(test_probs[it,true_labels[it].int()]),-100]))
    return res/test_data_len


def adaptive_calibration_error(test_probs,true_labels, test_data_len, num_classes,num_bins=20):
    o=torch.tensor(0.0).detach()
    for k in range(num_classes):
        ind=torch.argsort(test_probs[:,k],stable=True)        
        testprobsk=test_probs[:,k]
        sorted_probs=testprobsk[ind]
        sorted_true_labels=true_labels[ind]

        true_label_is_k = (sorted_true_labels==k).clone().detach().float()
        bins=(torch.tensor(range(test_data_len))/torch.tensor(test_data_len/num_bins)).floor()

        for b in range(num_bins):
            mask = (bins == b)
            if torch.any(mask):
                o += (true_label_is_k[mask] - sorted_probs[mask]).mean().abs()

    return o / (num_bins*num_classes)


In [ ]:
bias_arr_full=torch.zeros([5,6,10000]).detach()
bias_arr_full_sd4=torch.zeros([5,6,10000,4]).detach()
bias_arr=torch.zeros([5,6]).detach()
bias_arr_sd4=torch.zeros([5,6,4]).detach()
bias_arr_sd=torch.zeros([5,6]).detach()
training_size=no_batches*batch_size
test_size=test_data_len
num_classes=10

In [ ]:
str_list=["SG_BAOAB","SMS_BAOAB","SG_UBU","SMS_UBU", "SG_EM"]

for mit in range(5):
    for it in range(-1,5):
        str=str_list[mit]
        filepath="output_fashion_multinomial_%s_h%d.pickle"%(str,it)
        num_epochs=int(400*(2.0**it))

        with open(filepath,"rb") as file:
            test_labels_arr,test_prob_arr, test_prob_arr2=pickle.load(file)           

            test_prob=torch.Tensor(test_prob_arr[:,:,(num_epochs//5):num_epochs]).mean(-1).reshape(test_size,num_classes)
            test_prob2=torch.Tensor(test_prob_arr2[:,:,(num_epochs//5):num_epochs]).mean(-1).reshape(test_size,num_classes)

            test_prob_correct_label=torch.Tensor([test_prob[t, test_labels_arr[t].int()] for t in range(test_size)])
            test_prob_correct_label2=torch.Tensor([test_prob2[t, test_labels_arr[t].int()] for t in range(test_size)])

            #bias_arr[mit,it+1]=(test_prob_correct_label-test_prob_correct_label2).abs().mean()
            bias_arr_full[mit,it+1,:]=test_prob_correct_label-test_prob_correct_label2
            for bit in range(4):
                test_prob=torch.Tensor(test_prob_arr[:,:,(num_epochs//5)*bit:(num_epochs//5)*(bit+1)]).mean(-1).reshape(test_size,num_classes)
                test_prob2=torch.Tensor(test_prob_arr2[:,:,(num_epochs//5)*bit:(num_epochs//5)*(bit+1)]).mean(-1).reshape(test_size,num_classes)

                test_prob_correct_label=torch.Tensor([test_prob[t, test_labels_arr[t].int()] for t in range(test_size)])
                test_prob_correct_label2=torch.Tensor([test_prob2[t, test_labels_arr[t].int()] for t in range(test_size)])

                bias_arr_full_sd4[mit,it+1,:,bit]=test_prob_correct_label-test_prob_correct_label2

            #bias_arr[mit,it+1]=(test_prob-test_prob2).abs().sum(dim=1).mean()
            #bias_arr[mit,it+1]=(torch.log(test_prob_correct_label)-torch.log(test_prob_correct_label2)).nanmean().abs()
            
for mit in range(5):
    for it in range(-1,5):
        bias_arr[mit,it+1]=(bias_arr_full[mit,(it+1):6,:].sum(dim=0)).abs().mean()
        for bit in range(4):
            bias_arr_sd4[mit,it+1,bit]=(bias_arr_full_sd4[mit,(it+1):6,:,bit].sum(dim=0)).abs().mean()
        bias_arr_sd[mit,it+1]=torch.std(torch.log2(bias_arr_sd4[mit,it+1,:]))/2
# print("Average bias in probability of true category", (test_prob_correct_label-test_prob_correct_label2).abs().mean())


In [ ]:
for mit in range(5):
    for it in range(-1,5):
        bias_arr[mit,it+1]=(bias_arr_full[mit,(it+1):6,:].sum(dim=0)).abs().mean()
        for bit in range(4):
            bias_arr_sd4[mit,it+1,bit]=(bias_arr_full_sd4[mit,(it+1):6,:,bit].sum(dim=0)).abs().mean()
        bias_arr_sd[mit,it+1]=torch.std(torch.log2(bias_arr_sd4[mit,it+1,:]))/2

In [ ]:
torch.log2(bias_arr)

In [ ]:
bias_arr_sd

In [ ]:
from scipy.io import savemat
filepath="results_fashion_multinomial_bias.mat"
mdic={"bias_arr":bias_arr.cpu().numpy(),"bias_arr_sd":bias_arr_sd.cpu().numpy()}
savemat(filepath,mdic)

In [ ]:
str_list=["SG_BAOAB","SMS_BAOAB","SG_UBU","SMS_UBU", "SG_EM"]
rps_arr=torch.zeros(5,6)
acc_arr=torch.zeros(5,6)
ace_arr=torch.zeros(5,6)
nll_arr=torch.zeros(5,6)
rps_arr4=torch.zeros(5,6,4)
acc_arr4=torch.zeros(5,6,4)
ace_arr4=torch.zeros(5,6,4)
nll_arr4=torch.zeros(5,6,4)
rps_arr_sd=torch.zeros(5,6)
acc_arr_sd=torch.zeros(5,6)
ace_arr_sd=torch.zeros(5,6)
nll_arr_sd=torch.zeros(5,6)
num_classes=10
test_prob4=torch.zeros(10000,10,4)
for mit in range(5):
    for it in range(-1,5):
        str=str_list[mit]
        filepath="output_fashion_multinomial_%s_h%d.pickle"%(str,it)


        num_epochs=int(400*(2.0**it))

        with open(filepath,"rb") as file:
            test_labels_arr,test_prob_arr, test_prob_arr2=pickle.load(file)           

            test_prob=torch.Tensor(test_prob_arr[:,:,(num_epochs//5):num_epochs]).mean(-1).reshape(test_size,num_classes)

            for bit in range(4):
                test_prob4[:,:,bit]=torch.Tensor(test_prob_arr[:,:,(num_epochs//5)*bit:(num_epochs//5)*(bit+1)]).mean(-1).reshape(test_size,num_classes)
              
        rps_arr[mit,it+1]=(rps_calc(test_prob, test_labels_arr, test_data_len,num_classes).mean())
        nll_arr[mit,it+1]=nll_calc(test_prob, test_labels_arr,test_data_len,num_classes)
        ace_arr[mit,it+1]=adaptive_calibration_error(test_prob, test_labels_arr, test_data_len,num_classes).mean()
        _, predictedt = torch.max(test_prob,1)
        acc_arr[mit,it+1]=(predictedt==test_labels_arr.reshape(1,test_size)).sum()/test_size
        for bit in range(4):
            rps_arr4[mit,it+1,bit]=rps_calc(test_prob4[:,:,bit], test_labels_arr,test_data_len,num_classes).mean()
            nll_arr4[mit,it+1,bit]=nll_calc(test_prob4[:,:,bit], test_labels_arr,test_data_len,num_classes)
            ace_arr4[mit,it+1,bit]=adaptive_calibration_error(test_prob4[:,:,bit], test_labels_arr, test_data_len,num_classes).mean()
            _, predictedt = torch.max(test_prob4[:,:,bit],1)
            acc_arr4[mit,it+1,bit]=(predictedt==test_labels_arr.reshape(1,test_size)).sum()/test_size
        rps_arr_sd[mit,it+1]=torch.std(rps_arr4[mit,it+1,:])/2
        nll_arr_sd[mit,it+1]=torch.std(nll_arr4[mit,it+1,:])/2
        ace_arr_sd[mit,it+1]=torch.std(ace_arr4[mit,it+1,:])/2
        acc_arr_sd[mit,it+1]=torch.std(acc_arr4[mit,it+1,:])/2

from scipy.io import savemat
filepath="results_fashion_multinomial.mat"
mdic={"rps_arr":rps_arr.cpu().numpy(),"rps_arr_sd":rps_arr_sd.cpu().numpy(),\
      "ace_arr":ace_arr.cpu().numpy(),"ace_arr_sd":ace_arr_sd.cpu().numpy(),\
      "acc_arr":acc_arr.cpu().numpy(),"acc_arr_sd":acc_arr_sd.cpu().numpy(),\
      "nll_arr":nll_arr.cpu().numpy(),"nll_arr_sd":nll_arr_sd.cpu().numpy()}
savemat(filepath,mdic)